# use this code to download pdfs mentioned in csv

In [ ]:
import pandas as pd

data = pd.read_csv(r'cleaned_company_pdf_urls.csv')

In [ ]:
display(data)

In [ ]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

data = pd.read_csv('cleaned_company_pdf_urls.csv')[0:3000]
# data = pd.read_csv('cleaned_company_pdf_urls.csv')

def download_pdf(url_comp_pdf_tuple):
    idx, comp_url, pdf_url = url_comp_pdf_tuple
    response = requests.get(pdf_url)
    if response.status_code == 200:
        filename = f"reports/downloaded_{idx}.pdf"
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"idx :{idx} Downloaded PDF {pdf_url} successfully.")
        return (idx, comp_url, pdf_url, filename)
    else:
        print(f"idx :{idx} Failed to download PDF {pdf_url}. Status code: {response.status_code}")
        return (idx, comp_url, pdf_url, None)

# Use ThreadPoolExecutor to download PDFs concurrently
with ThreadPoolExecutor() as executor:
    url_comp_pdf_tuples = [(idx, comp_url, pdf_url) for idx, comp_url, pdf_url in data[['idx', 'comp_url', 'pdf_url']].itertuples(index=False)]
    results = executor.map(download_pdf, url_comp_pdf_tuples)

# Update the DataFrame with downloaded filename
downloaded_files = []
for result in results:
    idx, comp_url, pdf_url, filename = result
    if filename:
        downloaded_files.append(filename)
        data.loc[data['idx'] == idx, 'downloaded_filename'] = filename

# Save the updated DataFrame to a CSV file
data.to_csv('final_idx_compUrl_pdfUrl_filename.csv', index=False)

print("PDFs downloaded successfully.")
